In [2]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
client_id = "6f8bb3e64c2944d586037b9907043601"
client_secret = "81f5f44660364831be8d9118ccb51785"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
def album_songs(uri):
    album = uri 
    spotify_albums[album] = {}
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album'] = [] 
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['id'] = []
    spotify_albums[album]['name'] = []
    spotify_albums[album]['uri'] = []
    #pull data on album tracks
    tracks = sp.album_tracks(album) 
    for n in range(len(tracks['items'])): 
        spotify_albums[album]['album'].append(artist_album_names[album_count]) 
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['uri'].append(tracks['items'][n]['uri'])

def audio_features(album):
    #Add new key-values to store audio features
    spotify_albums[album]['acousticness'] = []
    spotify_albums[album]['danceability'] = []
    spotify_albums[album]['energy'] = []
    spotify_albums[album]['instrumentalness'] = []
    spotify_albums[album]['liveness'] = []
    spotify_albums[album]['loudness'] = []
    spotify_albums[album]['speechiness'] = []
    spotify_albums[album]['tempo'] = []
    spotify_albums[album]['valence'] = []
    spotify_albums[album]['popularity'] = []
    
    track_count = 0
    for track in spotify_albums[album]['uri']:
        #pull audio features per track
        features = sp.audio_features(track)
        
        #Append to relevant key-value
        spotify_albums[album]['acousticness'].append(features[0]['acousticness'])
        spotify_albums[album]['danceability'].append(features[0]['danceability'])
        spotify_albums[album]['energy'].append(features[0]['energy'])
        spotify_albums[album]['instrumentalness'].append(features[0]['instrumentalness'])
        spotify_albums[album]['liveness'].append(features[0]['liveness'])
        spotify_albums[album]['loudness'].append(features[0]['loudness'])
        spotify_albums[album]['speechiness'].append(features[0]['speechiness'])
        spotify_albums[album]['tempo'].append(features[0]['tempo'])
        spotify_albums[album]['valence'].append(features[0]['valence'])
        #popularity is stored elsewhere
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [4]:
genres = {'Qawwali': ['Nusrat Fateh Ali Khan', 'Abida Parveen', 'Sabri Brothers'], 'Ghazal': ['Mehdi Hassan', 'Jagjit Singh', 'Noor Jehan'], 'Sindhi': ['Allan Faqir', 'Zarina Baloch', 'Sanam Marvi'],
            'Pop': ['Ed Sheeran', 'Taylor Swift', 'Michael Jackson'], 'Rock': ['David Bowie', 'Freddie Mercury', 'Dave Grohl'], 'EDM': ['Skrillex', 'Avicci', 'David Guetta'],
            'Jazz': ['Louis Armstrong', 'Miles Davis', 'Billie Holiday'], 'Classical': ['Wolfgang Amadeus Mozart', 'Ludwig van Beethoven', 'Frederic Chopin'], 'Blues': ['B.B. King', 'Muddy Waters', 'Robert Johnson'],
            'Hip-Hop': ['Eminem', 'Jay-Z', 'Tupac Shakur']}

final_df = pd.DataFrame()

for v in genres:
    result = sp.search(genres[v]) 
    result['tracks']['items'][1]['artists']
    
    #Extract Artist's url
    artists_uris = result['tracks']['items'][0]['artists'][0]['uri']
    #Pull all of the artist's albums
    artist_albums = sp.artist_albums(artists_uris, album_type='album')
    #Store artist's albums' names' and uris in separate lists
    artist_album_names = []
    artist_album_uris = []
    for i in range(len(artist_albums['items'])):
        artist_album_names.append(artist_albums['items'][i]['name'])
        artist_album_uris.append(artist_albums['items'][i]['uri'])
        
    spotify_albums = {}
    album_count = 0
    for i in artist_album_uris: #each album
        album_songs(i)
        album_count+=1 #Updates album count once all tracks have been added

    import time
    import numpy as np
    sleep_min = 2
    sleep_max = 5
    start_time = time.time()
    request_count = 0
    for i in spotify_albums:
        audio_features(i)
        request_count+=1
        if request_count % 5 == 0:
            print(str(request_count) + " playlists completed")
            time.sleep(np.random.uniform(sleep_min, sleep_max))
            print('Loop #: {}'.format(request_count))
            print('Elapsed Time: {} seconds'.format(time.time() - start_time))

    dic_df = {}
    dic_df['album'] = []
    dic_df['track_number'] = []
    dic_df['id'] = []
    dic_df['name'] = []
    dic_df['uri'] = []
    dic_df['acousticness'] = []
    dic_df['danceability'] = []
    dic_df['energy'] = []
    dic_df['instrumentalness'] = []
    dic_df['liveness'] = []
    dic_df['loudness'] = []
    dic_df['speechiness'] = []
    dic_df['tempo'] = []
    dic_df['valence'] = []
    dic_df['popularity'] = []
    for album in spotify_albums: 
        for feature in spotify_albums[album]:
            dic_df[feature].extend(spotify_albums[album][feature])
            
    len(dic_df['album'])

    dataframe = pd.DataFrame.from_dict(dic_df)
    dataframe['Genre'] = v

    final_df = final_df.append(dataframe)

5 playlists completed
Loop #: 5
Elapsed Time: 16.006316423416138 seconds
10 playlists completed
Loop #: 10
Elapsed Time: 29.314232349395752 seconds
15 playlists completed
Loop #: 15
Elapsed Time: 43.69741201400757 seconds
20 playlists completed
Loop #: 20
Elapsed Time: 54.987908363342285 seconds


C:\Users\Noman\AppData\Local\Temp\ipykernel_15184\3220944448.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(dataframe)


5 playlists completed
Loop #: 5
Elapsed Time: 10.475309133529663 seconds
10 playlists completed
Loop #: 10
Elapsed Time: 19.687623262405396 seconds
15 playlists completed
Loop #: 15
Elapsed Time: 46.87960481643677 seconds
20 playlists completed
Loop #: 20
Elapsed Time: 73.11091899871826 seconds


C:\Users\Noman\AppData\Local\Temp\ipykernel_15184\3220944448.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(dataframe)
C:\Users\Noman\AppData\Local\Temp\ipykernel_15184\3220944448.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(dataframe)


5 playlists completed
Loop #: 5
Elapsed Time: 19.242437601089478 seconds
10 playlists completed
Loop #: 10
Elapsed Time: 43.50836801528931 seconds
15 playlists completed
Loop #: 15
Elapsed Time: 60.162999391555786 seconds


...

5 playlists completed
Loop #: 5
Elapsed Time: 21.62386131286621 seconds
10 playlists completed
Loop #: 10
Elapsed Time: 41.043909788131714 seconds


C:\Users\Noman\AppData\Local\Temp\ipykernel_15184\3220944448.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(dataframe)


5 playlists completed
Loop #: 5
Elapsed Time: 26.30845808982849 seconds
10 playlists completed
Loop #: 10
Elapsed Time: 46.076120138168335 seconds
15 playlists completed
Loop #: 15
Elapsed Time: 64.14825081825256 seconds
20 playlists completed
Loop #: 20
Elapsed Time: 92.94428277015686 seconds


C:\Users\Noman\AppData\Local\Temp\ipykernel_15184\3220944448.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(dataframe)


TypeError: 'NoneType' object is not subscriptable

In [7]:
final_df.to_csv('Spotify_music.csv')

In [5]:
final_df['Genre'].value_counts()

Rock       490
Jazz       432
Ghazal     348
Pop        299
Qawwali    235
EDM        232
Sindhi      24
Name: Genre, dtype: int64